In [326]:
## Om de code van BSARD te kunnen volgen moet de Dutch dataset hetzelfde format hebben als de French version. 
#Df_questions moet 'article_ids', question
#df_a heeft 'article'

In [1]:
import pandas as pd


ART_COLUMNS = ['id', 'reference', 'article', 'law_type', 'code', 'book', 'part', 'act', 'chapter', 'section', 'subsection', 'description']

# LOAD DATA

In [328]:
du_questions = pd.read_csv('../../translated_and_filtered/du_questions.csv')
du_articles = pd.read_csv('../../translated_and_filtered/du_articles.csv')
du_questions_synth = pd.read_csv('../../translated_and_filtered/du_synth_questions.csv')
len(du_questions), len(du_articles), len(du_questions_synth)

(1048, 20876, 104380)

In [329]:
fr_questions_test = pd.read_csv('../../fr_questions_test.csv')
fr_questions_train = pd.read_csv('../../fr_questions_train.csv')
fr_articles = pd.read_csv('../../fr_articles.csv')
fr_questions_synth = pd.read_csv('../../fr_questions_synthetic.csv')
len(fr_questions_test), len(fr_questions_train), len(fr_articles), len(fr_questions_synth)

(222, 886, 22633, 113165)

In [330]:
## Check if there are no duplicate id's
set(du_articles[du_articles.duplicated(subset=['french_id'], keep=False)]['french_id'])

set()

# ARTICLES

## Create French dataset

In [331]:
len(du_articles), len(fr_articles)

(20876, 22633)

In [332]:
#Only the articles that are present in the dutch dataset
fr_articles = fr_articles[fr_articles['id'].isin(du_articles['french_id'])]
len(fr_articles)

20876

In [333]:
#Create new ascending id's
fr_articles['new_id'] = range(1, len(fr_articles) + 1)
fr_articles

id                                          reference  \
0          1  Art. 1.1.1, Code Bruxellois de l'Air, du Clima...   
1          2  Art. 1.1.2, Code Bruxellois de l'Air, du Clima...   
2          3  Art. 1.2.1, Code Bruxellois de l'Air, du Clima...   
3          4  Art. 1.3.1, Code Bruxellois de l'Air, du Clima...   
4          5  Art. 1.4.1, Code Bruxellois de l'Air, du Clima...   
...      ...                                                ...   
22628  22629              Art. 194, La Constitution (Titre VII)   
22629  22630             Art. 195, La Constitution (Titre VIII)   
22630  22631             Art. 196, La Constitution (Titre VIII)   
22631  22632             Art. 197, La Constitution (Titre VIII)   
22632  22633             Art. 198, La Constitution (Titre VIII)   

                                                 article  law_type  \
0      Le présent Code règle une matière visée à l'ar...  regional   
1      Le présent Code transpose en Région de Bruxell...  regional   
2      Le présent Code poursuit les objectifs suivant...  regional   
3      Au sens du présent Code, il faut entendre par ...  regional   
4      Le plan régional Air-Climat-énergie, ci-après ...  regional   
...                                                  ...       ...   
22628  La ville de Bruxelles est la capitale de la Be...   federal   
22629  Le pouvoir législatif fédéral a le droit de dé...   federal   
22630  Aucune révision de la Constitution ne peut êtr...   federal   
22631  Pendant une régence, aucun changement ne peut ...   federal   
22632  D'un commun accord avec le Roi, les Chambres c...   federal   

                                                    code  \
0      Code Bruxellois de l'Air, du Climat et de la M...   
1      Code Bruxellois de l'Air, du Climat et de la M...   
2      Code Bruxellois de l'Air, du Climat et de la M...   
3      Code Bruxellois de l'Air, du Climat et de la M...   
4      Code Bruxellois de l'Air, du Climat et de la M...   
...                                                  ...   
22628                                    La Constitution   
22629                                    La Constitution   
22630                                    La Constitution   
22631                                    La Constitution   
22632                                    La Constitution   

                        book part                                 act  \
0      Dispositions communes  NaN                         Généralités   
1      Dispositions communes  NaN                         Généralités   
2      Dispositions communes  NaN                           Objectifs   
3      Dispositions communes  NaN                         Définitions   
4      Dispositions communes  NaN    Plan régional air-climat-énergie   
...                      ...  ...                                 ...   
22628                    NaN  NaN             Dispositions generales.   
22629                    NaN  NaN  De la revision de la constitution.   
22630                    NaN  NaN  De la revision de la constitution.   
22631                    NaN  NaN  De la revision de la constitution.   
22632                    NaN  NaN  De la revision de la constitution.   

                         chapter section subsection  \
0                            NaN     NaN        NaN   
1                            NaN     NaN        NaN   
2                            NaN     NaN        NaN   
3                            NaN     NaN        NaN   
4      Contenu et portée du plan     NaN        NaN   
...                          ...     ...        ...   
22628                        NaN     NaN        NaN   
22629                        NaN     NaN        NaN   
22630                        NaN     NaN        NaN   
22631                        NaN     NaN        NaN   
22632                        NaN     NaN        NaN   

                                             description  new_id  
0                     Dispositions com

## Dutch version in same format

In [334]:
def concatenate_reference(row):
    reference = f"{row['art_nr']}, {row['codex']} ("
    for column_name, cell_value in row.iteritems():
        if (column_name in ['part', 'book', 'title', 'chapter', 'section', 'subsection']) and not pd.isna(cell_value):
            reference += cell_value + ', '
    reference = reference[:-2] if reference.endswith(', ') else reference
    reference += ")"
    return reference
    
def concatenate_description(row):
    description = ""
    for column_name, cell_value in row.iteritems():
        if (column_name in ['part_name', 'book_name', 'title_name', 'chapter_name', 'section_name', 'subsection_name']) and not pd.isna(cell_value):
            description += cell_value + ', '
    description = description[:-2] if description.endswith(', ') else description
    return description

du_articles['reference'] = du_articles.apply(concatenate_reference, axis=1)
du_articles['description'] = du_articles.apply(concatenate_description, axis=1)

In [335]:
du_articles.sort_values(by='french_id', inplace=True)
fr_articles.sort_values(by='id', inplace=True)
du_articles['new_id'] = fr_articles['new_id'].values
du_articles['law_type'] = fr_articles['law_type'].values
du_articles

french_id     art_nr                                            article  \
486           1  Art.1.1.1  Onderhavig Wetboek regelt een materie als bedo...   
487           2  Art.1.1.2  Onderhavig Wetboek zet de volgende richtlijnen...   
488           3  Art.1.2.1  Onderhavig Wetboek streeft de volgende doelste...   
489           4  Art.1.3.1  In de zin van onderhavig Wetboek dient men te ...   
490           5  Art.1.4.1  Het gewestelijk Lucht-Klimaat-Energieplan, hie...   
...         ...        ...                                                ...   
8177      22629    Art.194  De stad Brussel is de hoofdstad van België en ...   
8178      22630    Art.195  De federale wetgevende macht heeft het recht t...   
8179      22631    Art.196  Er mag geen herziening van de Grondwet worden ...   
8180      22632    Art.197  Tijdens een regentschap mag in de Grondwet gee...   
8181      22633    Art.198  In overeenstemming met de Koning kunnen de gro...   

                                            subdivision  \
486   Ordonnantie houdende het Brussels Wetboek van ...   
487   Ordonnantie houdende het Brussels Wetboek van ...   
488   Ordonnantie houdende het Brussels Wetboek van ...   
489   Ordonnantie houdende het Brussels Wetboek van ...   
490   Ordonnantie houdende het Brussels Wetboek van ...   
...                                                 ...   
8177      De Gecoördineerde Grondwet, , TITEL VII, , ,    
8178     De Gecoördineerde Grondwet, , TITEL VIII, , ,    
8179     De Gecoördineerde Grondwet, , TITEL VIII, , ,    
8180     De Gecoördineerde Grondwet, , TITEL VIII, , ,    
8181     De Gecoördineerde Grondwet, , TITEL VIII, , ,    

                                                  codex part  \
486   Ordonnantie houdende het Brussels Wetboek van ...  NaN   
487   Ordonnantie houdende het Brussels Wetboek van ...  NaN   
488   Ordonnantie houdende het Brussels Wetboek van ...  NaN   
489   Ordonnantie houdende het Brussels Wetboek van ...  NaN   
490   Ordonnantie houdende het Brussels Wetboek van ...  NaN   
...                                                 ...  ...   
8177                         De Gecoördineerde Grondwet  NaN   
8178                         De Gecoördineerde Grondwet  NaN   
8179                         De Gecoördineerde Grondwet  NaN   
8180                         De Gecoördineerde Grondwet  NaN   
8181                         De Gecoördineerde Grondwet  NaN   

                               book                     title      chapter  \
486   BOEK 1. - ALGEMENE BEPALINGEN  TITEL 1. - Algemeenheden          NaN   
487   BOEK 1. - ALGEMENE BEPALINGEN  TITEL 1. - Algemeenheden          NaN   
488   BOEK 1. - ALGEMENE BEPALINGEN                   TITEL 2          NaN   
489   BOEK 1. - ALGEMENE BEPALINGEN                   TITEL 3          NaN   
490   BOEK 1. - ALGEMENE BEPALINGEN                   TITEL 4  HOOFDSTUK 1   
...                             ...                       ...          ...   
8177                            NaN                 TITEL VII          NaN   
8178                            NaN                TITEL VIII          NaN   
8179                            NaN                TITEL VIII          NaN   
8180                            NaN                TITEL VIII          NaN   
8181                            NaN                TITEL VIII          NaN   

     section  ... part_name book_name                             title_name  \
486      NaN  ...       NaN       NaN                                    NaN   
487      NaN  ...       NaN       NaN                                    NaN   
488      NaN  ...       NaN       NaN                         Doelstellingen   
489      NaN  ...       NaN       NaN                             Definities   
490      NaN  ...       NaN       NaN  Gewestelijk lucht-klimaat-energieplan   
...      ...  ...       ...       ...                                    ...   
8177     NaN  ...       NaN       NaN                    A

In [336]:
du_articles_data = du_articles[['new_id', 'reference', 'article', 'law_type', 'codex', 'book_name', 'part_name', 'title_name', 'chapter_name', 'section_name', 'subsection_name', 'description']]
du_articles_data.columns = ART_COLUMNS
fr_articles_data = fr_articles[['new_id', 'reference', 'article', 'law_type', 'code', 'book', 'part', 'act', 'chapter', 'section', 'subsection', 'description']]
fr_articles_data.columns = ART_COLUMNS
du_articles_data.info(), fr_articles_data.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 20876 entries, 486 to 8181
Data columns (total 12 columns):
 #   Column       Non-Null Count  Dtype 
---  ------       --------------  ----- 
 0   id           20876 non-null  int64 
 1   reference    20876 non-null  object
 2   article      20876 non-null  object
 3   law_type     20876 non-null  object
 4   code         20876 non-null  object
 5   book         16234 non-null  object
 6   part         9218 non-null   object
 7   act          18921 non-null  object
 8   chapter      18819 non-null  object
 9   section      12287 non-null  object
 10  subsection   4496 non-null   object
 11  description  20876 non-null  object
dtypes: int64(1), object(11)
memory usage: 2.1+ MB
<class 'pandas.core.frame.DataFrame'>
Int64Index: 20876 entries, 0 to 22632
Data columns (total 12 columns):
 #   Column       Non-Null Count  Dtype 
---  ------       --------------  ----- 
 0   id           20876 non-null  int64 
 1   reference    20876 non-null 

(None, None)

# QUESTIONS

## Refer to new id

In [337]:
def get_dutch_ids(row):
    article_ids = row['article_ids'].split(',')
    dutch_ids = []
    for art_id in article_ids:
        dutch_id = du_articles[du_articles['french_id']==int(art_id)]['new_id']
        if len(dutch_id) == 1:
            dutch_id = dutch_id.values[0]
        else:
            print("No id found for art " + str(art_id))
        dutch_ids.append(dutch_id)
    return ','.join(map(str, dutch_ids))

du_questions['new_article_ids'] = du_questions.apply(get_dutch_ids, axis=1)

In [338]:
fr_questions = pd.concat([fr_questions_test, fr_questions_train], ignore_index=True)
fr_questions = fr_questions[fr_questions['id'].isin(du_questions['id'])]
fr_questions.sort_values(by='id', inplace=True)
du_questions.sort_values(by='id', inplace=True)
fr_questions['new_article_ids'] = du_questions['new_article_ids'].values
len(fr_questions)

1048

In [339]:
du_questions = du_questions.drop(columns=['article_ids'])
fr_questions = fr_questions.drop(columns=['article_ids'])
du_questions.columns = ['id', 'category', 'subcategory', 'question', 'extra_description', 'article_ids']
fr_questions.columns = ['id', 'category', 'subcategory', 'question', 'extra_description', 'article_ids']

In [340]:
du_questions['category'] = du_questions['category'].replace('Zilver', 'Geld')

## New train_test split

In [341]:
from sklearn.model_selection import train_test_split
du_q_train_data, du_q_test_data = train_test_split(du_questions, test_size=0.2, random_state=42)
len(du_q_train_data), len(du_q_test_data)

(838, 210)

In [342]:
fr_q_train_data = fr_questions[fr_questions['id'].isin(du_q_train_data['id'])].set_index('id').loc[du_q_train_data['id']].reset_index()
fr_q_test_data = fr_questions[fr_questions['id'].isin(du_q_test_data['id'])].set_index('id').loc[du_q_test_data['id']].reset_index()
len(fr_q_train_data), len(fr_q_test_data)

(838, 210)

In [343]:
du_q_train_data.info(), fr_q_train_data.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 838 entries, 566 to 213
Data columns (total 6 columns):
 #   Column             Non-Null Count  Dtype 
---  ------             --------------  ----- 
 0   id                 838 non-null    int64 
 1   category           838 non-null    object
 2   subcategory        838 non-null    object
 3   question           838 non-null    object
 4   extra_description  738 non-null    object
 5   article_ids        838 non-null    object
dtypes: int64(1), object(5)
memory usage: 45.8+ KB
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 838 entries, 0 to 837
Data columns (total 6 columns):
 #   Column             Non-Null Count  Dtype 
---  ------             --------------  ----- 
 0   id                 838 non-null    int64 
 1   category           838 non-null    object
 2   subcategory        838 non-null    object
 3   question           838 non-null    object
 4   extra_description  738 non-null    object
 5   article_ids        838 non-nu

(None, None)

In [363]:
du_q_train_data.iloc[1, 3]

'Is het altijd nodig om een medische keuring te ondergaan voordat je weer aan het werk gaat?'

In [372]:
du_articles

french_id     art_nr                                            article  \
486           1  Art.1.1.1  Onderhavig Wetboek regelt een materie als bedo...   
487           2  Art.1.1.2  Onderhavig Wetboek zet de volgende richtlijnen...   
488           3  Art.1.2.1  Onderhavig Wetboek streeft de volgende doelste...   
489           4  Art.1.3.1  In de zin van onderhavig Wetboek dient men te ...   
490           5  Art.1.4.1  Het gewestelijk Lucht-Klimaat-Energieplan, hie...   
...         ...        ...                                                ...   
8177      22629    Art.194  De stad Brussel is de hoofdstad van België en ...   
8178      22630    Art.195  De federale wetgevende macht heeft het recht t...   
8179      22631    Art.196  Er mag geen herziening van de Grondwet worden ...   
8180      22632    Art.197  Tijdens een regentschap mag in de Grondwet gee...   
8181      22633    Art.198  In overeenstemming met de Koning kunnen de gro...   

                                            subdivision  \
486   Ordonnantie houdende het Brussels Wetboek van ...   
487   Ordonnantie houdende het Brussels Wetboek van ...   
488   Ordonnantie houdende het Brussels Wetboek van ...   
489   Ordonnantie houdende het Brussels Wetboek van ...   
490   Ordonnantie houdende het Brussels Wetboek van ...   
...                                                 ...   
8177      De Gecoördineerde Grondwet, , TITEL VII, , ,    
8178     De Gecoördineerde Grondwet, , TITEL VIII, , ,    
8179     De Gecoördineerde Grondwet, , TITEL VIII, , ,    
8180     De Gecoördineerde Grondwet, , TITEL VIII, , ,    
8181     De Gecoördineerde Grondwet, , TITEL VIII, , ,    

                                                  codex part  \
486   Ordonnantie houdende het Brussels Wetboek van ...  NaN   
487   Ordonnantie houdende het Brussels Wetboek van ...  NaN   
488   Ordonnantie houdende het Brussels Wetboek van ...  NaN   
489   Ordonnantie houdende het Brussels Wetboek van ...  NaN   
490   Ordonnantie houdende het Brussels Wetboek van ...  NaN   
...                                                 ...  ...   
8177                         De Gecoördineerde Grondwet  NaN   
8178                         De Gecoördineerde Grondwet  NaN   
8179                         De Gecoördineerde Grondwet  NaN   
8180                         De Gecoördineerde Grondwet  NaN   
8181                         De Gecoördineerde Grondwet  NaN   

                               book                     title      chapter  \
486   BOEK 1. - ALGEMENE BEPALINGEN  TITEL 1. - Algemeenheden          NaN   
487   BOEK 1. - ALGEMENE BEPALINGEN  TITEL 1. - Algemeenheden          NaN   
488   BOEK 1. - ALGEMENE BEPALINGEN                   TITEL 2          NaN   
489   BOEK 1. - ALGEMENE BEPALINGEN                   TITEL 3          NaN   
490   BOEK 1. - ALGEMENE BEPALINGEN                   TITEL 4  HOOFDSTUK 1   
...                             ...                       ...          ...   
8177                            NaN                 TITEL VII          NaN   
8178                            NaN                TITEL VIII          NaN   
8179                            NaN                TITEL VIII          NaN   
8180                            NaN                TITEL VIII          NaN   
8181                            NaN                TITEL VIII          NaN   

     section  ... part_name book_name                             title_name  \
486      NaN  ...       NaN       NaN                                    NaN   
487      NaN  ...       NaN       NaN                                    NaN   
488      NaN  ...       NaN       NaN                         Doelstellingen   
489      NaN  ...       NaN       NaN                             Definities   
490      NaN  ...       NaN       NaN  Gewestelijk lucht-klimaat-energieplan   
...      ...  ...       ...       ...                                    ...   
8177     NaN  ...       NaN       NaN                    A

## Synthetic questions

In [346]:
#Keep only questions that are in dutch dataset
fr_questions_synth = fr_questions_synth[fr_questions_synth['id'].isin(du_questions_synth['id'])]
len(fr_questions_synth), len(du_questions_synth)

(104380, 104380)

In [347]:
du_questions_synth = du_questions_synth.merge(du_articles[['french_id', 'new_id']], left_on='article_ids', right_on='french_id', how='left')[['id', 'question', 'new_id']]
len(du_questions_synth)

104380

In [352]:
fr_questions_synth = fr_questions_synth.merge(du_questions_synth[['id', 'new_id']], left_on='id', right_on='id', how='left')[['id', 'question', 'new_id']]
len(fr_questions_synth)

In [353]:
# fr_questions_synth = fr_questions_synth[['id', 'question', 'new_id']]

In [354]:
fr_questions_synth.columns = ['id', 'question', 'article_ids']
du_questions_synth.columns = ['id', 'question', 'article_ids']

# SAVE DATA

In [286]:
du_articles_data.to_csv('../../new_data/dutch/du_articles.csv', index=False)
fr_articles_data.to_csv('../../new_data/french/fr_articles.csv', index=False)
du_q_train_data.to_csv('../../new_data/dutch/du_train.csv', index=False)
fr_q_train_data.to_csv('../../new_data/french/fr_train.csv', index=False)
du_q_test_data.to_csv('../../new_data/dutch/du_test.csv', index=False)
fr_q_test_data.to_csv('../../new_data/french/fr_test.csv', index=False)
du_questions_synth.to_csv('../../new_data/dutch/du_synth.csv', index=False)
fr_questions_synth.to_csv('../../new_data/french/fr_synth.csv', index=False)

In [355]:
du_questions_synth.to_csv('../../new_data/dutch/du_synth.csv', index=False)
fr_questions_synth.to_csv('../../new_data/french/fr_synth.csv', index=False)

# HELP

In [2]:
du_art = pd.read_csv('../../new_data/dutch/du_articles.csv')
fr_art = pd.read_csv('../../new_data/french/fr_articles.csv')
len(du_art), len(fr_art)


(20876, 20876)

In [380]:
df_check[df_check['article'].apply(lambda x: len(x)>40000)]

id                                          reference  \
11091  11092  Art.R.IV.1-1, Waalse Wetboek van Ruimtelijke O...   
11092  11093  Art.R.IV.1-1_DUITSTALIGE_GEMEENSCHAP, Waalse W...   
12691  12692   Art.VII.N3, Economisch Recht (BOEK VII, TITEL 7)   
20666  20667  Art.211bis, Wetboek diverse rechten en taksen ...   

                                                 article  law_type  \
11091  Handelingen, werken en installaties vrijgestel...  regional   
11092  Handelingen, werken en installaties vrijgestel...  regional   
12691  Bijlage 3. - Europees gestandaardiseerd inform...   federal   
20666  § 1. Dit artikel legt de voorschriften en proc...   federal   

                                              code  \
11091  Waalse Wetboek van Ruimtelijke Ontwikkeling   
11092  Waalse Wetboek van Ruimtelijke Ontwikkeling   
12691                             Economisch Recht   
20666            Wetboek diverse rechten en taksen   

                                                    book part  \
11091         Stedenbouwkundige vergunningen en attesten  NaN   
11092         Stedenbouwkundige vergunningen en attesten  NaN   
12691                      BETALINGS- EN KREDIETDIENSTEN  NaN   
20666  Bepalingen gemeen aan de diverse rechten en ta...  NaN   

                                          act    chapter section subsection  \
11091                                Algemeen  Begrippen     NaN        NaN   
11092                                Algemeen  Begrippen     NaN        NaN   
12691                          Slotbepalingen        NaN     NaN        NaN   
20666  Bepalingen gemeen aan alle belastingen        NaN     NaN        NaN   

                                             description  
11091  Stedenbouwkundige vergunningen en attesten, Al...  
11092  Stedenbouwkundige vergunningen en attesten, Al...  
12691      BETALINGS- EN KREDIETDIENSTEN, Slotbepalingen  
20666  Bepalingen gemeen aan de diverse rechten en ta...

In [378]:
df_check = pd.read_csv('../../new_data/dutch/du_articles.csv')

In [390]:
# Assuming 'df_check' is your DataFrame and 'article' is the column containing the text
df_check['article'] = df_check['article'].str.split('Handelingen/werken/ installaties').str.get(0)
df_check['article'] = df_check['article'].str.split('(Inleidende tekst)').str.get(0)
# Print the updated DataFrame
df_check[df_check['article'].apply(lambda x: len(x)>40000)]


id                                          reference  \
20666  20667  Art.211bis, Wetboek diverse rechten en taksen ...   

                                                 article law_type  \
20666  § 1. Dit artikel legt de voorschriften en proc...  federal   

                                    code  \
20666  Wetboek diverse rechten en taksen   

                                                    book part  \
20666  Bepalingen gemeen aan de diverse rechten en ta...  NaN   

                                          act chapter section subsection  \
20666  Bepalingen gemeen aan alle belastingen     NaN     NaN        NaN   

                                             description  
20666  Bepalingen gemeen aan de diverse rechten en ta...

In [394]:
# df_check[df_check['article'].str.split('(Inleidende tekst)').apply(len)>1]
len(df_check[df_check['article'].apply(lambda x: len(x)>10000)])

53

In [391]:
df_check.to_csv('../../new_data/dutch/du_articles.csv', index=False)

In [392]:
len(df_check)

20876